### 기초 예제
#### 1. 이미지 입력

``` python
cv2.imread(
    fileName,
    flags = cv2.IMREAD_COLOR
)
```

기본 플래그 : 8비트 3채녈, BGR => ((height, width, 3), np.uint8)

In [3]:
import numpy as np
import cv2

src = cv2.imread("OpenCV_Logo.png", cv2.IMREAD_GRAYSCALE)
# cv2.IMREAD_GRAYSCALE : 단일 채널 그레이스케일로 이미지 변환
print(src.ndim, src.shape, src.dtype)

2 (739, 600) uint8


#### 2. 이미지 출력
``` python
cv2.imshow(
    winname,
    ndarray
)
```

In [4]:
cv2.namedWindow("src", flags=cv2.WINDOW_FREERATIO)  # 윈도우 생성(윈도우 양식 설정, 기본값: None)
# cv2.WINDOW_FREERATIO : 비율의 제한이 없는 경우 이미지를 최대한 확장
cv2.resizeWindow("src", 400, 200)   # 윈도우 크기 설정, 출력하는 이미지의 크기와 관련 없음.
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyWindow("src")

``` python
# 마우스 콜백 설정 함수
cv2.setMouseCallback(
    windowName,
    onMouse,
    param = None    # 사용자 정의 데이터(param)은 모든 형식의 데이터를 전달할 수 있음. (원본 이미지, List, Tuple) 
)
# 마우스 콜백 함수
def fun_name(
    event,
    x,
    y,
    flags,
    param
)
```

In [5]:
def mouse_event(event, x, y, flags, param):
    global radius
    if event == cv2.EVENT_LBUTTONDOWN:  # 마우스를 누를때
        cv2.circle(param, (x, y), radius, (255, 0, 0), 2)
        cv2.imshow("draw", src)

    elif event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            radius += 1
        elif radius > 1:
            radius -= 1

radius = 3
src = np.full((500, 500, 3), 255, dtype=np.uint8)

cv2.imshow("draw", src)
cv2.setMouseCallback("draw", mouse_event, src)
cv2.waitKey()
cv2.destroyAllWindows()

#### 3. 동영상 입력

In [6]:
capture = cv2.VideoCapture("C:\\Source\\openCV\\basic-openCV\\images\\Star.mp4")

while True:
    ret, frame = capture.read() # ret(bool)은 정상적으로 프레임을 읽었는지, frame(ndarray) 현재 프레임

    # 현재 프레임의 수 == 총 프레임 수
    if(capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
        capture.open("C:\\Source\\openCV\\basic-openCV\\images\\Star.mp4")

    cv2.imshow("VideoFrame", frame)
    if cv2.waitKey(33) == ord('q'): break   # 프레임당 33ms => 약 30FPS
    # capture.get(cv2.CAP_PROP_FPS)로 동영상 파일의 프레임을 알 수 이음
capture.release()   # 동영상 파일 닫고 메모리 해제
cv2.destroyAllWindows()

#### 4. 카메라 출력

#### 5. 이미지 연결

``` python
# 수평 이미지 연결 함수
dst = cv2.hconcat(
    src
)

# 수직 이미지 연결함수
dst = cv2.vconcat(
    src
)

```

In [7]:
one = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\one.jpg")
two = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\two.jpg")
three = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\three.jpg")
four = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\four.jpg")


horizontal1 = np.full((50, one.shape[1], 3), [0,0,0], dtype=np.uint8)
horizontal2 = np.full((50, two.shape[1], 3), [0,0,0], dtype=np.uint8)

left = cv2.vconcat((one, horizontal1, three))
#left = np.vstack((one, horizontal1, three))
#right = cv2.vconcat((two, horizontal2, three))
right = np.vstack((two, horizontal2, four)) # Numpy함수를 이용해서도 이미지 변경 가능(배열 병합 및 분리)

vertical = np.full((left.shape[0], 50, 3), 0, dtype=np.uint8)

dst = cv2.hconcat((left, vertical, right))
# dst = np.hstatck((left, vertical, right))
# dst = np.concatenate((left, line, right), axis=1)

cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

#### 6. 도형 그리기

- 위치, 두께, 색상, 선형 타입, 비트 시프트

1. 선형 타입
    - **브레젠험 알고리즘(Bresenham's algorithm) 방식**  
    일반적으로 직선의 방성식에 의한 좌표는 실수 형태.  
    but, 이미지는 래스터 형식의 사각형 격자 구조의 행렬, 점의 점표는 모두 정수의 값  
    => 실수 연산을 하지 않고 정수 연산으로만 선을 그릴 수 있도록 개발한 알고리즘
        - 4연결 방식: 선분의 위치를 보고, 오른쪽, 왼쪽 위쪽, 아래쪽 영역 고려
        - 8연결 방식: 선분의 위치를 보고, 대각선 방향까지 추가 (총 8개의 위치 고려)

    - **안티 에일리어싱(Anti-Aliasing) 방식**  
    영상의 결함을 없애기 위한 기법  
    계단 현상을 없애고 부드럽게 보이게 하는 방식  
    가우스 필터링을 사용, 넓은 선의 경우 항상 둥글게 그려짐.  

    - 내부 채우기 방식

2. 비트 시프트  
도형 그리기 함수에서 사용되는 값은 일반적으로 정수값.  
but, 비트 시프트를 활용하면 실숫값 좌표로도 도형 그리기 함수를 사용할 수 있음.  
서브 픽셀(sub pixel) 정렬을 지원하여 소수점 이하 자릿수를 표현  

    오른쪽 시프트 연산으로 간주해서 이해(마지막에 반올림)  
0100(2) -> 0010(2)

    - 0-shift
    (2,2) (8,5) 선분
    - 1-shift
    (2,2) (8,5) -> (1,1) (4, 2.5) -> (1,1) (4,3) 선분
    - 2-shift
    (2,2) (8,5) -> (0.5,0.5) (2,1.25) -> (1,1) (2,1) 선분

##### 1.직선 그리기
``` python
cv2.line(
    img,
    pt1,    # 시작 좌표
    pt2,    # 도착 좌표
    color,  # BGR형식
    thickness = None,
    lineType = None,    # 선형 타입
    shift = None    # 비트 시프트, 실숫값으로 처리할 좌표(pt)의 비트 값
)
```

In [8]:
img = np.zeros((768, 1366, 3), dtype = np.uint8)

cv2.line(img, (100, 100), (1200, 100), (0, 0, 255), 3, cv2.LINE_AA)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

##### 2. 사각형 그리기
``` python
cv2.rectangle(
    img,
    pt1,    # 좌측 상단 모서리 좌표
    pt2,    # 우측 하단 모서리 좌표
    color,  # BGR형식
    thickness = None,
    lineType = None,
    shift = None
)
```

In [9]:
cv2.rectangle(img, (500, 200), (1000, 400), (255, 0, 0), 5, cv2.LINE_8)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


##### 3. 원 그리기
``` python
cv2.circle(
    img,
    center,    # 원의 중심
    radius,    # 반지름
    color,  # BGR형식
    thickness = None,
    lineType = None,
    shift = None
)
```

In [10]:
cv2.circle(img, (300, 300), 50, (0, 255, 0), cv2.FILLED, cv2.LINE_4)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


##### 4. 호 그리기
``` python
cv2.ellipse(
    img,
    center,    # 원의 중심
    axes,      # 장축과 단축
    angle,     # 각도(장축으로부터 기울어진 각도)
    startAngle, # 시작각도
    endAngle,    # 도착 각도
    color,  # BGR형식
    thickness = None,
    lineType = None,
    shift = None
)
```

In [11]:
cv2.ellipse(img, (1200, 300), (100,  50), 0, 50, 180, (255, 255, 0), 2)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


##### 5. 내부가 채워지지 않은 다각형 그리기
``` python
cv2.FillPoly(
    img,
    pts,    # 선들의 묶음
    isClosed
    color,  # BGR형식
    lineType = None,
    shift = None
    offset = None
)
```

In [12]:
pts1 = np.array([[[100, 500], [300, 500], [200, 600]], [[400, 500], [500, 500], [600, 700]]])   # 동일한 꼭짓점 개수를 갖는 다각형만 가능
cv2.polylines(img, pts1, True, (0, 255, 255), 2)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


##### 6. 내부가 채워진 다각형 그리기
``` python
cv2.polylines(
    img,
    pts,    # 선들의 묶음
    color,  # BGR형식
    lineType = None,
    shift = None
    offset = None
)
```

In [13]:
pts2 = np.array([[700, 500], [800, 500], [700, 600]])
cv2.fillPoly(img, [pts2], (255, 0, 255), cv2.LINE_AA)


cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

##### 7. 문자 그리기
``` python
cv2.putText(
    img,
    text,
    org,    # 텍스트 박스의 좌측 상단 모서리 기준
    fontFace, # 글꼴
    fontScale,  # 글자 크기# 그래픽스나 문자열 입력값에서 사용하는 글자 크기 단위 아님. 글꼴도
    color,
    thickness = None,
    lineType = None,
    bottomLeftOrigin = None # 텍스트 박스의 좌측 하단 모서리를 사용할경우 true로 지정
)
```

In [14]:
cv2.putText(img, "OpenCV", (900, 600), cv2.FONT_HERSHEY_COMPLEX | cv2.FONT_ITALIC, 2, (255, 255, 255), 3)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 7. 트랙 바
일정 범위 내의 값을 변경할때 주로 사용, 적절한 임계값을 찾거나 변경하기 위해 사용

``` python
# 트랙 바 생성 함수
cv2.createTrackbar(
    trackbarName,
    windowName,
    value,
    count,  # 최대값
    onChange    # 콜백 함수 # c#에서와 달리 필수임
)
```

``` python
# 트랙 바 위치 반환 함수
retval = cv2.getTrackbarPos(
    trackbarName,
    windowName
)
```

In [15]:
def onChangeBlue(pos):
    global b
    b = pos
    cv2.imshow("Palette", createImage(b,g,r))

def createImage(b, g, r):
    return np.full((500, 500, 3), (b, g, r), dtype = np.uint8)

b, g, r = 0, 0, 0
cv2.namedWindow("Palette")
cv2.createTrackbar("Blue", "Palette", 55, 255, onChangeBlue)
cv2.createTrackbar("Green", "Palette", 0, 255, lambda x:x)        # 콜백함수를 사용하지 않을 때는 익명함수(람수 함수)를 활용할 수 있음
cv2.createTrackbar("Red", "Palette", 0, 255, lambda x:x)       

while True:
    g = cv2.getTrackbarPos("Green", "Palette")
    r = cv2.getTrackbarPos("Red", "Palette")

    cv2.imshow("Palette", createImage(b,g,r))
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

#### 이미지 저장

``` python
cv2.imwrite(
    fileaname,
    img,
    params = None
)
```

In [16]:
img = np.zeros((480, 640, 3), dtype = np.uint8)

save = cv2.imwrite("CV.jpeg", img, (cv2.IMWRITE_JPEG_QUALITY, 200, cv2.IMWRITE_JPEG_PROGRESSIVE, 1))
print(save)

True


#### 동영상 저장
``` python
# 동영상 저장 구조 생성
capture.open(
    filename,
    foucc,  # 동영상 파일을 저장할 때 사용할 압축 코덱(디지털 포맷 코드)
    fps,
    frameSize,
    isColor = true

 # 동영상 파일에 플레임을 저장
videoWriter.write(image)   
)
```

In [17]:
capture = cv2.VideoCapture("C:\\Source\\openCV\\basic-openCV\\images\\Star.mp4")
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
videoWriter = cv2.VideoWriter() # 녹화를 위한 메로리를 할당
isWrite = False

while True:
    ret, frame = capture.read()

    if(capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
        capture.open("C:\\Source\\openCV\\basic-openCV\\images\\Star.mp4")

    cv2.imshow("VideoFrame", frame)  
    key = cv2.waitKey(33)      

    if key == 4:
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        videoWriter.open("Video.avi", fourcc, 30, (width, height), True)   
        isWrite = True
    elif key == 24:
        videoWriter.release()
        isWrite = False

    elif key == ord('q'): break

    if isWrite == True:
        videoWriter.write(frame)    # 동영상 파일에 플레임을 저장

videoWriter.release()   # 동영상 저장 구조 메모리 해제
capture.release()
cv2.destroyAllWindows()
